## Descrição do Notebook

Neste notebook, os dados são agregados por dia e as regras aplicadas estão descritas ao decorrer do código.

In [1]:
import pandas as pd 
import numpy as np
import xlrd             #lê dados do excel
import matplotlib.pyplot as plt


### Lendo o arquivo e informando que a tabela DATA_TRAT. é do tipo datetime

In [2]:
df = pd.read_csv("../datasets/df_rpo_v3.csv")

df['DATA_TRAT.'] = pd.to_datetime(df["DATA_TRAT."], dayfirst = True)


### Atribuindo apenas dados referentes a 2017 à variável df

In [3]:
df = df.loc[(df['DATA_TRAT.'] >= '2017-01-01') & (df['DATA_TRAT.'] <= '2017-12-31')]

Retirando P10 e P90 das vendas dos produtos:

Criando o *Dataframe* agregado por ano, mes, dia

In [4]:
# Criando colunas de Semana e Ano para auxiliar na agregação
df.reset_index(inplace = True, drop = True)
#df["Semana"] = df['DATA_TRAT.'].dt.isocalendar().week
df["Ano"] = df["DATA_TRAT."].dt.year
df["Mes"] = df['DATA_TRAT.'].dt.month
df["Dia"] = df['DATA_TRAT.'].dt.day

> Fazendo as agregações: 
* Colunas de **Vendas Reais, Margem e CMV** foram somadas
* Colunas de **Preço  e Porcentagem de Margem** foram calculadas com base na média
* Coluna da razão **CMV/QTD** foi retirada uma nova fração entre a coluna CMV e QTD para respeitar a razão entre as grandezas.

In [5]:
### Definindo df_Agg
df_agg = df[["SAP+NOME", "UF", "Ano","Mes", "Dia" , "preço2", "Mg %",
                     "Margem Líquida Real %", "Preço"]]
### Definindo df_Agg_sum
df_agg_sum = df[["SAP+NOME", "UF", "Mes", "Dia", "Vendas Reais Qtde", "Vendas Reais $",
                     "Margem Líquida Real $", "Venda Líquida Real $", "CMV"]]

In [6]:
### Agrupando df_agg por SAP+NOME, UF e Semana e calculando a média
df_agg = df_agg.groupby(["SAP+NOME", "UF", "Mes", "Dia"]).mean().reset_index()

### Agrupando df_Agg_sum por SAP+NOME, UF e Semana e somando as demais variáveis
df_agg_sum = df_agg_sum.groupby(["SAP+NOME", "UF", "Mes", "Dia"]).sum().reset_index()

### Calculando CMV individual
df_agg_sum["CMV/QTD"] = df_agg_sum["CMV"] / df_agg_sum["Vendas Reais Qtde"]

Unindo os dois *dataframes* de agregação criados para chegar ao *dataset* que será depositado no *Data Lake*

In [7]:
### Criando df_final como cópia do df_Agg
df_final = df_agg.copy()

### Fazendo merge entre df_final e df_agg_sum utilizando SAP+NOME, UF e Semana como Chaves
df_final = df_final.merge(df_agg_sum, on = ["SAP+NOME", "UF", "Mes", "Dia"], how = "inner")

### Resetando index
df_final.reset_index(inplace = True, drop = True)

### Verificação de dados duplicados

In [8]:
df_final.duplicated(subset=['SAP+NOME', 'UF', 'Mes', 'Dia']).value_counts()

False    58864
dtype: int64

Retirando *P10* e *P90* dos produtos que possuiram variações de preço mais de 10 vezes. Essa agregação foi feita por **Produto** e por **UF** para eliminar os preços promocionais e as sazonalidades de vendas causadas por eles. E para as demais situações de um dado **Produto** ou **UF**, onde o preço não variou mais de 10 vezes, mas, ainda sim, há sazonalidade na quantidade de vendas semanais, também foram removidos os valores menores que *P10*, para remover quantidades insignificantes e *P90*, para remover valores que destoam.

In [9]:
lst_uf = df_final["UF"].unique()
lst_sap = df_final["SAP+NOME"].unique()

for uf in lst_uf:
    for sap in lst_sap:
        df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
        if df_seg.empty == True:
               continue
        else:
            # Se preço variar mais de 10 vezes, encontraremos o p10 e o p90 de valor e quantidade
            if len(df_seg["Preço"].unique()) > 10:
                q10_p = np.quantile(df_seg["Preço"].unique(), .10)
                q90_p = np.quantile(df_seg["Preço"].unique(), .90)
                q_10_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .10)
                q_90_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .90)
                
                # Atribuindo a 'ind', todos os valores que atendem as condições
                ind = df_seg.loc[((df_seg["Preço"] < q10_p)  | (df_seg["Preço"] > q90_p)) &
                                 ((df_seg["Vendas Reais Qtde"] < q_10_qtd) | (df_seg["Vendas Reais Qtde"] > q_90_qtd))]
                
                # Dropando index dos valores de 'ind'
                df_final = df_final.drop(ind.index)
            
            # Atribuindo a df_seg o valor de df_final ao final da condição
            df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
            
            # Pegando o p10 e o p90 da coluna Vendas Reais Qtde
            p10 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .10)
            p90 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .90)
            
            # Atribuindo a variável ind todos os index que são menores que o p10 e maiores que o p90
            ind = df_seg.loc[(df_seg["Vendas Reais Qtde"] < p10)  | (df_seg["Vendas Reais Qtde"] > p90)]
            
            # Removendo do DF todos os index que foram atribuidos a variável ind
            df_final = df_final.drop(ind.index)

### Ordenando o df final por Ano, Mes, Dia e UF

In [10]:
df_final = df_final.sort_values(by = ["Ano", "Mes", "Dia", "UF"]).reset_index(drop = True)

In [11]:
df_final

,SAP+NOME,UF,Mes,Dia,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,Vendas Reais $,Margem Líquida Real $,Venda Líquida Real $,CMV,CMV/QTD
0,000000000004243420 - AMACIANTE COMFORT CONCENT...,AL,1,2,2017.0,7.99,0.407011,40.701149,7.990,12.0,95.88,35.41,87.00,51.59,4.299167
1,000000000004243420 - AMACIANTE COMFORT CONCENT...,AM,1,2,2017.0,7.99,0.554937,55.493749,7.865,24.0,188.76,104.75,188.76,84.01,3.500417
2,000000000004243420 - AMACIANTE COMFORT CONCENT...,AP,1,2,2017.0,7.99,0.499373,49.937312,7.990,11.0,87.89,39.83,79.76,39.93,3.630000
3,000000000004263231 - JG CAMA QUEEN PRATA 150FI...,AP,1,2,2017.0,129.99,0.498731,49.873123,129.990,1.0,129.99,47.17,94.58,47.41,47.410000
4,000000000002158151 - T VISITA P BORDA DESIRE A...,BA,1,2,2017.0,4.99,0.416721,41.672131,4.190,10.0,41.90,12.71,30.50,17.79,1.779000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34494,000000000004243420 - AMACIANTE COMFORT CONCENT...,TO,12,31,2017.0,4.99,0.005587,0.558747,4.990,35.0,174.65,0.71,127.07,126.36,3.610286
34495,000000000004248118 - AMACIANTE DOWNY LIRIOS DO...,TO,12,31,2017.0,6.99,0.178437,17.843683,6.990,12.0,83.88,10.89,61.03,50.14,4.178333
34496,000000000004261082 - KIT CABIDE C/3 MADEIRA REV,TO,12,31,2017.0,1.99,0.146507,14.650706,10.990,14.0,153.86,16.40,111.94,95.54,6.824286
34497,000000000004266282 - JG CAMA QUEEN MICROFIBRA ...,TO,12,31,2017.0,49.99,0.277426,27.742645,49.990,2.0,99.98,20.18,72.74,52.56,26.280000


### Gravando df_final como arquivo CSV

In [12]:
df_final.to_csv("../datasets/df_rpo_v4_2_dia.csv", index = False)